In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU in use:", torch.cuda.get_device_name(0))
else:
    print("CUDA is NOT available. Using CPU.")

CUDA is NOT available. Using CPU.


In [2]:
# Importing gemma7b model
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils.quantization_config import BitsAndBytesConfig

# c2t model loading example
import whisper_s2t
from whisper_s2t.backends.ctranslate2.model import BEST_ASR_CONFIG

#preprocessing audio
import importlib
import split_and_preprocess
importlib.reload(split_and_preprocess)
from split_and_preprocess import process_chunks

'ffmpeg' is not built with soxr resampler, using 'swr' resampler. This may degrade performance.


e:\anaconda3\envs\mals2t\lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
# determine the device
device = "cuda" if torch.cuda.is_available() else "cpu"

### Loading the whisper model

In [ ]:
ct2_model = whisper_s2t.load_model(
    model_identifier="large-v2", 
    backend='CTranslate2', 
    compute_type='int8',  # Best for cpu
    device=device,
    asr_options=BEST_ASR_CONFIG,
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

### Loading Gemma7b

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
llm_model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it")
llm_model = llm_model.to(device)

#### Preprocessing

In [ ]:
output_path = "output.wav"
audio_path = "test.wav"

# Process and merge audio chunks
merged_audio, sr = process_chunks(audio_path, output_path, chunk_duration=30)

### Translation using whisper model

In [ ]:
lang_codes = ['ml']
initial_prompts = [None]
# Direct Malayalam-to-English translation
tasks = ['translate']

output_path = "output.wav"
files = [output_path]
    
ct2_out = ct2_model.transcribe_with_vad(
        files,
        lang_codes=lang_codes,
        tasks=tasks,
        initial_prompts=initial_prompts,
        batch_size=16
    )

    # cleaned_segments = [
    #     seg.text for seg in segments
    #     if getattr(seg, "avg_logprob", None) is not None and seg.avg_logprob > -1.0 and getattr(seg, "no_speech_prob", 1.0) < 0.5
    # ]
    # cleaned_transcript = " ".join(cleaned_segments)

print("c2t English translation:")
print(ct2_out[0][0]['text'])   # English text

### Further refining translated text using another llm

In [ ]:
# Craft a prompt with instructions and context
input_text = (
    "You are an expert translator who can translate malayalam to english. "
    "Given the following sentence, make it more sensible and fluent in English. "
    "If the sentence does not make sense, use the words in the sentence to form the most plausible translation that makes sense. "
    "The translation must be such that if its translated back to malayalam, we get a similar sentence as the original malayalam sentence. "
    "Here is the sentence:\n"
    f"{ct2_out[0][0]['text']}\n"
    "Improved translation:"
)
# Generate the improved translation
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = llm_model.generate(**input_ids)
improved_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(tokenizer.decode(outputs[0]))